In [1]:
\l p.q

In [2]:
seq:.p.import[`keras;`:models;`:Sequential;*]
keras:.p.import `keras
dense:.p.import[`keras;`:layers;`:Dense;*]
K:.p.import[`keras;`:backend]
pd:.p.import `pandas
np:.p.import `numpy
hvd:.p.import `horovod.tensorflow

/opt/A3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/opt/A3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
hvd[`:init;*][]

{[f;x]embedPy[f;x]}[foreign]enlist


In [4]:
nparray:.p.import[`numpy;`:array;>]

In [5]:
colStr:"IIIII"
d:(colStr;enlist ",")0: `:/home/kkumar/q/l64/hdata.csv

In [6]:
cd:cols d

In [7]:
e:"f"$d`Event;t:"f"$d`Time

In [8]:
x:"f"$flip d cd where not cd in `Event`Time

In [9]:
hvd[`:init;*]
tf:.p.import `tensorflow

k){'[y;x]}/,[enlist]|:@[{[f;x]embedPy[f;x]}].[code[code;]enlist[;;][foreign]]..


In [10]:
trainnet:{[data]
    nn:tf[`:layers;`:dense;*;data;3;`activation pykw tf[`:nn;`:relu]];
    nn:tf[`:layers;`:dense;*;nn;100;`activation pykw tf[`:nn;`:relu]];
    nn:tf[`:layers;`:dense;*;nn;100;`activation pykw tf[`:nn;`:relu]];
    nn:tf[`:layers;`:dense;*;nn;100;`activation pykw tf[`:nn;`:relu]];
    nn:tf[`:layers;`:dense;*;nn;1;`activation pykw tf[`:nn;`:relu]];
    :nn
    }

In [11]:
nn:trainnet[tf[`:convert_to_tensor;*;nparray x til 1000]]

In [12]:
ev:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`e)]
tm:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`t)]
xv:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`x)]

In [13]:
cost:tf[`:divide;*;tf[`:subtract;*;tf[`:transpose;*;nn];tf[`:log;*;tf[`:cumsum;*;tf[`:exp;*;nn]]]];tf[`:reduce_sum;*;e]];

In [14]:
opt:tf[`:train;`:GradientDescentOptimizer;*;0.01*hvd[`:size;<][]]

In [15]:
global_step:tf[`:train;`:get_or_create_global_step;*][]

In [16]:
dop:hvd[`:DistributedOptimizer;*;opt]
opt:dop[`:minimize;*;cost;`global_step pykw global_step]

In [17]:
sess:K[`:get_session][]
sess[`:run;*;tf[`:global_variables_initializer;*][]]

2018-03-12 10:03:56.117375: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


{[f;x]embedPy[f;x]}[foreign]enlist


In [18]:
p)import tensorflow as tf
p)import horovod.tensorflow as hvd
p)config = tf.ConfigProto()
p)config.gpu_options.allow_growth=True
p)config.gpu_options.visible_device_list=str(hvd.local_rank())
config:.p.get`config

In [19]:
K[`:set_session;*;tf[`:Session;*;`config pykw config]]

{[f;x]embedPy[f;x]}[foreign]enlist


In [20]:
tf[`:add_to_collection;*;`prediction;nn]
p)l=list()
sz:count d

{[f;x]embedPy[f;x]}[foreign]enlist


In [21]:
mon_sess:tf[`:Session;*;`config pykw config]
mon_sess[`:run;*;[tf[`:global_variables_initializer;*][]]]

{[f;x]embedPy[f;x]}[foreign]enlist


In [22]:
i:0;
.p.set[`mon_sess;mon_sess]
.p.set[`opt;opt]
.p.set[`cost;cost]
.p.set[`x;xv]
.p.set[`e;ev]
while[i<sz;
    if[1000<=(sz-i);
        .p.set[`X;nparray x til 1000];
        .p.set[`E;nparray e til 1000];
        o:.p.eval"mon_sess.run([tf.get_collection(\"prediction\"),opt,cost],feed_dict={x:X,e:E})";
        .p.set[`o;o];
        .p.eval"l.append(o[0])"];
    i:i+1000;
    ]
    

In [23]:
l:.p.get`l

In [25]:
l: l`

In [30]:
count raze over l

5064000


In [31]:
l:(count d)#(raze over l)

In [32]:
l

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.887201 0 0 0 0.691890..
